In [1]:
import numpy as np
import pandas as pd

import mohusets.fuzzynumbers as fns
import mohusets.fuzzysets as fs
import mohusets.aggregators as agg

from utils import show_result

In [2]:
def HPFWBM(alter,weight=None,p=1,q=1):
    def bonferroni_element(d1,d2,p_,q_):
        s1 = fns.algeb_multiply(d1.algeb_power(p_),d2.algeb_power(q_))
        s2 = fns.algeb_multiply(d1.algeb_power(q_),d2.algeb_power(p_))
        return fns.algeb_plus(s1,s2)

    dlist = []
    if weight is None:
        dl = alter
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(bonferroni_element(d1,d2,p,q))
    else:
        dl = []
        assert len(alter) == len(weight),'The number of alter is not equal to the number of weight!'
        for i in range(len(alter)):
            dl.append(alter[i].algeb_times(weight[i]))
        for i,d1 in enumerate(dl):
            for j,d2 in enumerate(dl[:i]):
                dlist.append(bonferroni_element(d1,d2,p,q))

    agge = fns.qrungdhfe(qrung,[0],[1])
    for agg in dlist:
        agge = fns.algeb_plus(agg,agge)
    agge = (agge.algeb_times(1/(len(alter)*(len(alter)-1)))).algeb_power(1/(p+q))
    return agge

def WHPFMSM(alter,weight=None,k=1):
    if k==1:
        return agg.sub_weighted_ave(alter,weight)
    elif k==2:
        return HPFWBM(alter,weight)
    else:
        return None

In [3]:
dataFile = 'A.csv'
dataPath = 'data/'+dataFile

qrung = 2

weight = [0.2,0.15,0.2,0.3,0.15]

t = fs.fuzzyset(qrung,'qrungdhfe')
t.loadcsv(dataPath)

In [4]:
t.mat

,0,1,2,3,4
0,"[[0.3, 0.5], [0.6]]","[[0.6], [0.4, 0.6]]","[[0.4], [0.2]]","[[0.5], [0.4]]","[[0.3], [0.6]]"
1,"[[0.8], [0.2]]","[[0.6, 0.7], [0.1]]","[[0.7], [0.3, 0.4]]","[[0.3], [0.2]]","[[0.6], [0.1]]"
2,"[[0.2], [0.3]]","[[0.3], [0.5]]","[[0.5, 0.8], [0.2]]","[[0.3], [0.7]]","[[0.5], [0.4, 0.5]]"


In [5]:
res1 = [WHPFMSM(t.set[0],weight,k=1).score,WHPFMSM(t.set[1],weight,k=1).score,WHPFMSM(t.set[2],weight,k=1).score]
res2 = [WHPFMSM(t.set[0],weight,k=2).score,WHPFMSM(t.set[1],weight,k=2).score,WHPFMSM(t.set[2],weight,k=2).score]

In [6]:
r1 = show_result(t.set,'Garg_WHPFMSM(k=1)',res1,reverse=True)
r2 = show_result(t.set,'Garg_WHPFMSM(k=2)',res2,reverse=True)
result = pd.concat([r1,r2])

In [7]:
result

,A1,A2,A3,Ranking Order
Garg_WHPFMSM(k=1),0.042120,0.371397,0.031817,A2>A1>A3
Garg_WHPFMSM(k=2),-0.677872,-0.419742,-0.676821,A2>A3>A1
